# Estruturas Criptográficas - Criptografia e Segurança da Informação

[Grupo 03](https://paper.dropbox.com/doc/Estruturas-Criptograficas-2023-2024-Trabalhos-Praticos-8WcsdZARGLv0nXS9KasmK)

(PG54177) Ricardo Alves Oliveira 

(PG54236) Simão Oliveira Alvim Barroso

## TP4 - Exercício 1

In [22]:
import time
from sage.all import *
import hashlib


In [23]:
def bits_to_integer(y):
    alpha = len(y)
    x = 0
    for i in range(1, alpha + 1):
        x = y[alpha - i] + 2 * x
    return x

def bits_to_bytes(y):
    c = len(y)
    num_bytes = ceil(c / 8)
    z = [0] * num_bytes
    for i in range(c):
        z[i // 8] = z[i // 8] + y[i] * 2**(i % 8)
    return z

def bit_reverse(x, bits):
    y = 0
    for i in range(bits):
        y = (y << 1) | (x & 1)
        x >>= 1
    return y

def bytes_to_bits(z):
    d = len(z)
    y = [0] * (d * 8)
    for i in range(d):
        for j in range(8):
            y[8*i + j] = z[i] % 2
            z[i] = z[i] // 2
    return y

def bitlen(x):
    return x.nbits()

def simple_bit_pack(w, b):
    z = []
    for i in range(256):
        z = z+integer_to_bits(w[i], bitlen(b))
    return bits_to_bytes(z)

def simple_bit_unpack(v, b):
    c = bitlen(b)
    z = bytes_to_bits(v)
    w = [0] * 256
    for i in range(256):
        # BitsToInteger((z[ic], z[ic + 1], . . . z[ic + c − 1]), c)
        w[i] = bits_to_integer(z[i*c:(i+1)*c])
    return w

def bit_pack(w, a, b):
    z = []
    for i in range(256):
        z = z+integer_to_bits(b - w[i], bitlen(a + b))
    return bits_to_bytes(z)

def bit_unpack(v, a, b):
    c = bitlen(a+b)
    z = bytes_to_bits(v)
    w = []
    for i in range(256):
        wi = b - bits_to_integer(z[i*c:(i+1)*c])
        w.append(wi)
    return w

def H1024(input_bytes):
    hash_output = hashlib.sha256(input_bytes).digest()
    concatenated_output = hash_output
    while len(concatenated_output) < 128:  
        hash_output = hashlib.sha256(hash_output).digest()
        concatenated_output += hash_output
    concatenated_output = concatenated_output[:128]  

    bit_array = []
    for byte in concatenated_output:
        bits = bin(byte)[2:].zfill(8) 
        bit_array.extend(int(bit) for bit in bits)

    return bit_array

def integer_to_bits(x, alpha):
    y = [0] * alpha
    for i in range(alpha):
        y[i] = Integer(x) % Integer(2)
        x = x // 2
    return y

def coef_from_three_bytes(b0, b1, b2, q):
    if b2 > 127:
        b2 -= 128 
    z = 2 ** 16 * b2 + 2 ** 8 * b1 + b0
    if z < q:
        return z
    else:
        return None

def coef_from_half_byte(b, n):
    if n == 2 and b < 15:
        return 2 - (b % 5)
    elif n == 4 and b < 9:
        return 4 - b
    else:
        return None
    
def rej_ntt_poly(seed, q):
    a_hat = [None] * 256
    j = 0
    c = 0
    while j < 256:
        a_hat[j] = coef_from_three_bytes(hashlib.sha256(seed).digest()[c%30],hashlib.sha256(seed).digest()[(c+1)%30],hashlib.sha256(seed).digest()[(c+2)%30],q)
        c += 3
        if a_hat[j] is not None:
            j += 1
    return a_hat

def rej_bounded_poly(seed, q, n):
    a = [0] * 256
    j = 0
    c = 0
    while j < 256:
        z = hashlib.sha256(bytearray(seed)).digest()[c%32]
        z0 = coef_from_half_byte(z % 16, n)
        z1 = coef_from_half_byte(z // 16, n)
        if z0 is not None:
            a[j] = z0
            j += 1
        if z1 is not None and j < 256:
            a[j] = z1
            j += 1
        c += 1
    return a

def expand_a(p, q, k, l):
    A_hat = [[None for _ in range(l)] for _ in range(k)] 
    for r in range(k):
        for s in range(l):
            bits_s = integer_to_bits(s, 8)
            bits_r = integer_to_bits(r, 8)
            combined_bytes = bytearray(p) + bytearray(bits_s) + bytearray(bits_r)
            A_hat[r][s] = rej_ntt_poly(combined_bytes, q)[s]
    return A_hat

def expand_s(p, l, k,q,n):
    s1 = [rej_bounded_poly(p + integer_to_bits(r, 16),q,n) for r in range(l)]
    s2 = [rej_bounded_poly(p + integer_to_bits(r + l, 16),q,n) for r in range(k)]
    return (s1, s2)

def polynomial_mul(a, b, q):
    return [(a[i] * b[i]) % q for i in range(len(a))]

def ntt(w, q):
    w_hat = [0 for _ in range(256)]
    for j in range(256):
        w_hat[j] = w[j]
    size = 256
    z=1753
    zeta_brvs = [pow(z, bit_reverse(k, 8), q) for k in range(size)]
    k = 0
    length = size // 2
    while length >= 1:
        start = 0
        while start < size:
            k += 1
            zeta = zeta_brvs[k % size] % q
            for j in range(start, start + length):
                t = zeta * w_hat[j + length] % q
                w_hat[j + length] = (w_hat[j] - t) % q
                w_hat[j] = (w_hat[j] + t) % q
            start += 2 * length
        length //= 2
    return w_hat

def ntt_inverse(w_hat, q):
    size = 256
    w = [0 for _ in range(256)]
    for j in range(256):
        w[j] = w_hat[j]
    z=1753
    zeta_brvs = [pow(z, bit_reverse(k, 8), q) for k in range(size)]
    k = size
    length = 1
    while length < size:
        start = 0
        while start < size:
            k -= 1
            zeta = (-zeta_brvs[k % size]) % q
            for j in range(start, start + length):
                t = w[j]
                w[j] = (t + w[j + length]) % q
                w[j + length] = (t - w[j + length]) % q
                w[j + length] = zeta * w[j + length] % q
            start += 2 * length
        length *= 2
    f = inverse_mod(size, q)
    for j in range(size):
        w[j] = f * w[j] % q
    return w

def mod_plus_minus(m, alpha):
    m_prime = Integer(m) % Integer(alpha)
    if m_prime > alpha // 2:
        m_prime -= alpha
    return m_prime

def power2_round(r, q, d):
    r_plus = Integer(r) % Integer(q)
    r0 = mod_plus_minus(r_plus,2**d)
    r1 = (r_plus - r0) // (2**d)
    return (r1, r0)

def pk_encode(p, t1, q, d, k):
    pk = bits_to_bytes(p)
    max_value = (2 ** (bitlen(q - 1) - d)) - 1
    for i in range(k):
        pk += simple_bit_pack(t1[i], max_value)
    return pk

def pk_decode(pk, q, d, k):
    y = pk[:32]
    len_z = 32*(bitlen(q - 1)-d)
    z=[]
    for i in range(k):
        z.append(pk[32+i*len_z:32+(i+1)*len_z])
    p = bytes_to_bits(y)
    max_value = (2 ** (bitlen(q - 1) - d)) - 1
    t = [None for _ in range(k)]
    for i in range(k):
        t[i] = simple_bit_unpack(z[i], max_value)
    return p, t

def sk_encode(p, K, tr, s1, s2, t0, d, n): 
    sk = bits_to_bytes(p) + bits_to_bytes(K) + bits_to_bytes(tr)
    for si in s1:
        sk = sk + bit_pack(si, n, n)
    for si in s2:
        sk = sk + bit_pack(si, n, n)
    for ti in t0:
        sk = sk + bit_pack(ti, (2**(d-1))-1, 2**(d-1))
    return sk

def sk_decode(sk, d, n, l, k):
    f = sk[:32]
    g = sk[32:64]
    h = sk[64:128]
    a_len = 32 * bitlen(2*n)
    y=[]
    for i in range(l):
        y.append(sk[128+i*a_len:128+(i+1)*a_len])
    z=[]
    for i in range(k):
        z.append(sk[128+l*a_len+i*a_len:128+l*a_len+(i+1)*a_len])
    w_len = 32*d
    w=[]
    for i in range(k):
        w.append(sk[128+(l+k)*a_len+i*w_len:128+(l+k)*a_len+(i+1)*w_len])
    p = bytes_to_bits(f)
    K = bytes_to_bits(g)
    tr = bytes_to_bits(h)
    s1 = [bit_unpack(yi, n, n) for yi in y]
    s2 = [bit_unpack(zi, n, n) for zi in z]
    t0 = [bit_unpack(wi, (2**(d-1))-1, 2**(d-1)) for wi in w]
    return p, K, tr, s1, s2, t0

In [24]:
def ML_DSA_KeyGen(Tq, k, l, q, d, n):
    # Step 1: Choose random seed
    eps = [randint(0, 1) for _ in range(256)]

    # Step 2: Expand seed
    H_output = H1024(bytearray(eps))
    p = H_output[:256]
    p_ = H_output[256:768]
    K = H_output[768:]

    print('p:',p)
    print('p_:',p_)
    print('k:',K)

    # Step 3: Expand A
    A_hat = expand_a(p, q, k, l)

    print('A_hat:',A_hat)

    # Step 4: Expand S
    s1, s2 = expand_s(p_, l, k,q,n)


    # Step 5: Compute t = As1 + s2

    A_NTT_s1 =[]

    for i in range(l):
        s1_poly = PolynomialRing(Zmod(q), 'x')(s1[i])
        ntt_s1 = ntt(s1_poly, q)
        A_NTT_s1.append(polynomial_mul(A_hat[i], ntt_s1, q))


    t = [ntt_inverse(PolynomialRing(Zmod(q), 'x')(A_NTT_s1_row), q) for A_NTT_s1_row in A_NTT_s1]
    for i in range(k):
        for j in range(256):
            t[i][j] = (t[i][j] + s2[i][j])

    print('t:',t)

    # Step 6: Compress t
    t1, t0 = [], []
    for tt in t:
        tt1, tt0 = [], []
        for ti in tt:
            t1i, t0i = power2_round(ti, q, d)
            tt1.append(t1i)
            tt0.append(t0i)
        t1.append(tt1)
        t0.append(tt0)

    print('t1:',t1)
    print('t0:',t0)

    # Step 7: pkEncode
    pk = pk_encode(p, t1, q, d,k)

    # Step 8: Compute tr
    tr = H1024(bytearray(pk))[:512]

    # Step 9: skEncode
    sk = sk_encode(p, K, tr, s1, s2, t0, d, n)
    

    return pk, sk

In [25]:
# Parameters
Tq = 128
k = 4
l = 4
q = 8380417
d = 13
n = 2
gamma_1 = 2**17
gamma_2 = (q - 1) // 88
omega = 80
beta = 79

pk, sk = ML_DSA_KeyGen(Tq, k, l, q, d, n)
print('pk:',pk)
print('sk:',sk)

p: [1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0]
p_: [0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1

In [26]:
def hint_bit_pack(h):
    k = len(h)
    omega = sum(1 for hi in h for coeff in hi if coeff == 1)
    y = [0] * (omega + k)
    index = 0
    for i in range(k):
        for j in range(256):
            if h[i][j] == 1: 
                y[index] = j
                index += 1
        y[omega + i] = index
    return y

def hint_bit_unpack(y, k):
    omega = len(y) - k
    h = [[0] * 256 for _ in range(k)]  
    index = 0
    for i in range(k):
        if y[omega + i] < index or y[omega + i] > omega:
            return None
        while index < y[omega + i]:
            h[i][y[index]] = 1
            index += 1
    while index < omega:
        if y[index] == 0:
            return None
        index += 1
    return h

def sig_encode(c_tilde, z, h, y1):
    σ = bits_to_bytes(c_tilde)
    for zi in z:
        σ += bit_pack(zi, y1 - 1, y1)
    σ += hint_bit_pack(h)
    return σ

def sig_decode(σ, y1):
    w, *x, y = bytes_to_bits(σ)
    c_tilde = bits_to_bytes(w)
    z = [bit_unpack(xi, y1 - 1, y1) for xi in x]
    h = hint_bit_unpack(y)
    return c_tilde, z, h

def w1_encode(w1, q, y2):
    w1_tilde = ()
    for wi in w1:
        w1_tilde += bytes_to_bits(simple_bit_pack(wi, (q - 1) / (2 * y2) - 1))
    return w1_tilde


def decompose(r,q,y2):
    r_plus = r % q
    r0 = r_plus % (2**y2)
    if r_plus - r0 == q - 1:
        r1 = 0
        r0 -= 1
    else:
        r1 = (r_plus - r0) // (2^y2)
    return (r1, r0)

def brv(k, num_bits):
    result = 0
    for i in range(num_bits):
        bit = (k >> i) & 1
        result |= bit << (num_bits - 1 - i)
    
    return result

def ntt_inverse(w_hat, q, eps):
    w = [0] * 256
    for j in range(256):
        w[j] = w_hat[j]
    k = 256
    len = 1
    while len < 256:
        start = 0
        while start < 256:
            k -= 1
            zeta = -eps * brv(k) % q
            for j in range(start, start + len):
                t = w[j]
                w[j] = t + w[j + len]
                w[j + len] = t - w[j + len]
                w[j + len] = zeta * w[j + len]
            start += 2 * len
        len *= 2
    f = 8347681  
    for j in range(256):
        w[j] = f * w[j]
    return w

def montgomery_reduce(a, q):
    QINV = 58728449 
    t = (a % 2^32) * QINV % 2^32
    r = (a - t * q) % q
    return r

def sample_in_ball(seed, tau, q):
    c = PolynomialRing(Zmod(q), 'x').zero()
    k = 8
    for i in range(256 - tau, 256):
        while hashlib.sha256(seed + str(k).encode()).digest()[0] > i:
            k += 1
        j = hashlib.sha256(seed + str(k).encode()).digest()[0] 
        c[j] = (-1) ** (hashlib.sha256(seed + str(i - 256).encode()).digest()[0])
        k += 1
    return c

def expand_mask(seed, mu, ell, gamma_1):
    c = [None] * ell
    for r in range(ell):
        n = '{0:016b}'.format(mu + r)
        v = [hashlib.sha256(seed + n.encode()).digest()[i] for i in range(32 * r, 32 * (r + 1))]
        c[r] = Integer(bitstring=v, signed=True).bitslice(0, gamma_1)
    return c

def high_bits(poly):
    return poly.coefficients(sparse=False)[0]

def low_bits(poly):
    return poly.coefficients(sparse=False)[-1]

def make_hint(z, r):
    r1 = high_bits(r)
    v1 = high_bits(r + z)
    return r1 == v1

def use_hint(h, r, q, gamma_2):
    m = (q - 1) // (2 * gamma_2)
    r1 = high_bits(r)
    r0 = low_bits(r)
    if h and r0 > 0:
        return (r1 + 1) % m
    elif h and r0 <= 0:
        return (r1 - 1) % m
    else:
        return r1

In [27]:
def ML_DSA_Sign(sk, M, q, d, gamma_1, gamma_2, omega, beta):
    rho, K, tr, s1, s2, t0 = sk_decode(sk)
    s1_hat = [ntt_inverse(poly, q, omega) for poly in s1]
    s2_hat = [ntt_inverse(poly, q, omega) for poly in s2]
    t0_hat = [ntt_inverse(poly, q, omega) for poly in t0]
    A_hat = expand_a(rho, q, omega)
    mu = hashlib.sha512(tr + M.encode()).digest()
    rho_prime = hashlib.sha512(K + b'0' + mu).digest()
    kappa = 0
    z, h = None, None
    while z is None or h is None:
        y = expand_mask(rho_prime, mu, len(s1_hat) + len(s2_hat), gamma_1)
        w = [ntt_inverse(A_hat[i] * ntt(poly, q, omega), q, omega) for i, poly in enumerate(y)]
        w1 = [high_bits(poly) for poly in w]
        c_tilde = hashlib.sha512(mu + hint_bit_pack(w1)).digest()
        c_tilde_1, c_tilde_2 = c_tilde[:256], c_tilde[256:]
        c = sample_in_ball(c_tilde_1, q)
        c_hat = ntt(c, q, omega)
        cs1_hat = [ntt_inverse(c_hat * s_hat, q, omega) for s_hat in s1_hat]
        cs2_hat = [ntt_inverse(c_hat * s_hat, q, omega) for s_hat in s2_hat]
        z = [y[i] + cs1_hat[i] for i in range(len(s1_hat))]
        r0 = [low_bits(w[i] - cs2_hat[i]) for i in range(len(s2_hat))]
        if max(map(max, z)) >= gamma_1 - beta or max(map(max, r0)) >= gamma_2 - beta:
            z, h = None, None
        else:
            ct0_hat = [ntt_inverse(c_hat * t_hat, q, omega) for t_hat in t0_hat]
            h = make_hint(-ct0_hat, [w[i] - cs2_hat[i] + ct0_hat[i] for i in range(len(s2_hat))])
            if max(map(max, ct0_hat)) >= gamma_2 or sum(map(sum, h)) > omega:
                z, h = None, None
        kappa += len(s1_hat)
    sigma = sig_encode(c_tilde, [poly % q for poly in z], h, len(y[0]))
    return sigma

In [28]:
# Step 2: Choose a message
M = "This is a test message."
M_bytes = bytearray(M.encode())
M_bits = bytes_to_bits(M_bytes)
print(M_bits)

# Step 3: Sign the message
sign = ML_DSA_Sign(sk, M, q, d, gamma_1, gamma_2, omega, beta)


[0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0]


TypeError: sk_decode() missing 4 required positional arguments: 'd', 'n', 'l', and 'k'

In [ ]:
def ML_DSA_Verify(pk, M, sigma, q, gamma_1, gamma_2, omega, beta):
    rho, t1 = pk
    c_tilde, z, h = sig_decode(sigma, len(t1[0]))
    if h is None:
        return False
    A_hat = expand_a(rho, q, omega)
    tr = hashlib.sha512(bytes_to_bits(pk)).digest()
    mu = hashlib.sha512(tr + M.encode()).digest()
    c_tilde_1, _ = c_tilde[:256], c_tilde[256:]
    c = sample_in_ball(c_tilde_1, q)
    w_prime = [ntt_inverse(poly, q, omega) for poly in z]
    w1_prime = [high_bits(poly) for poly in w_prime]
    c_tilde_prime = hashlib.sha512(mu + hint_bit_pack(w1_prime)).digest()
    c_tilde_prime_1, _ = c_tilde_prime[:256], c_tilde_prime[256:]
    c_prime = sample_in_ball(c_tilde_prime_1, q)
    c_prime_hat = ntt(c_prime, q, omega)
    w_hat = [ntt(A_hat[i] * ntt_inverse(c_prime_hat, q, omega), q, omega) for i in range(len(t1))]
    w_hat = [use_hint(h, w_hat[i], q, gamma_2) for i in range(len(t1))]
    c_tilde_prime_check = hashlib.sha512(mu + hint_bit_pack([high_bits(poly) for poly in w_hat])).digest()
    return c_tilde == c_tilde_prime_check


In [ ]:
# Step 4: Verify the signature
is_valid = ML_DSA_Verify(pk, M, sign, q, gamma_1, gamma_2, omega, beta)

# Step 5: Print the result
if is_valid:
    print("Signature is valid.")
else:
    print("Signature is invalid.")


ValueError: too many values to unpack (expected 6)

In [ ]:
# Mensagem de exemplo
message = "Exemplo de mensagem para assinatura"

# Geração de chaves
public_key, private_key, keygen_time = generate_keys()
print(f"Tempo de geração de chaves: {keygen_time:.4f} segundos")

# Produção da assinatura
signature, sign_time = sign(message, private_key)
print(f"Tempo de produção da assinatura: {sign_time:.4f} segundos")

# Verificação da assinatura
verification, verify_time = verify(message, signature, public_key)
print(f"Tempo de verificação da assinatura: {verify_time:.4f} segundos")
print(f"A assinatura é válida? {'Sim' if verification else 'Não'}")


Tempo de geração de chaves: 0.0055 segundos
Tempo de produção da assinatura: 0.0003 segundos
Tempo de verificação da assinatura: 0.0002 segundos
A assinatura é válida? Não
